# Aula 07 - Filtragem Baseada em Conhecimento - Exercícios

## Importação dos dados (MovieLens 100k)

In [26]:
import pandas as pd
import numpy as np

In [27]:
import wget
!python3 -m wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip


100% [......................................................] 4924029 / 4924029
Saved under ml-100k.zip
Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-

In [28]:
#Types of genres
genre = pd.read_csv('./ml-100k/u.genre', sep="|", encoding='latin-1', header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [29]:
#Types of occupations
occupation = pd.read_csv('./ml-100k/u.occupation', sep="|", encoding='latin-1', header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [30]:
#Load the Ratings data
data = pd.read_csv('./ml-100k/u.data', sep="\t", header=None)
data.columns = ['userId', 'movieId', 'rating', 'timestamp']
data.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [31]:
#Load the Movies data
item = pd.read_csv('./ml-100k/u.item', sep="|", encoding='latin-1', header=None)
item.columns = ['movieId', 'title' ,'release','video release date', 'IMDb URL', 'unknown', 'Action', 
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item['release'] = pd.to_datetime(item['release'])
item = item[pd.notnull(item['release'])]
item['year'] = item['release'].dt.year.astype(int)
item.drop(columns=['release', 'video release date', 'IMDb URL'], inplace=True)
item.head()

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995


In [32]:
df_meta = item.melt(id_vars=['movieId', 'title'], var_name='genre')
df_meta = df_meta[df_meta.value == 1]
df_meta.drop(columns=['value'], inplace=True)
# df_meta[df_meta['movieId']==1]
df_meta.head()

,movieId,title,genre
1371,1373,Good Morning (1971),unknown
1682,2,GoldenEye (1995),Action
1684,4,Get Shorty (1995),Action
1697,17,From Dusk Till Dawn (1996),Action
1701,21,Muppet Treasure Island (1996),Action


In [33]:
# Obter a lista de gêneros de um item
def get_genres(df, movieId):
    if movieId not in df['movieId'].values:
        return []
    return df.loc[(df.movieId==movieId),'genre'].tolist()

get_genres(df_meta, 1)

['Animation', "Children's", 'Comedy']

In [34]:
#Load the User data
user = pd.read_csv('./ml-100k/u.user', sep="|", encoding='latin-1', header=None)
user.columns = ['userId', 'age', 'gender', 'occupation', 'zip code']
user.head()

,userId,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


***Exercício 01:*** Implemente um sistema de recomendação baseado em casos que permite as seguintes restrições definidas pelo usuário:
- Escolher entre filmes mais antigos ou mais recentes
- Determinar os gêneors de maior preferência (mas não restrito a eles)
- Escolher os filmes mais populares ou os menos populares dentre os usuários de uma dada profissão.
- Atribuir importância maior ou menor para cada uma das restrições anteriores.

Recomende uma lista de 10 filmes com base nos critérios definidos pelo usuário.

Exemplo: usuário define:
- filmes_mais_recentes = True
- generos_preferidos = ['Drama']
- filmes_mais_populares = True
- profissão = 'writer'
- pesos = [0.33, 0.33, 0.33]

Genero: tirar a similaridade jaccard entre a lista de generos providenciada e a lista de generos de um cado filme

Recencia de filme: determinar distancia entre data atual e filme. Se forem filmes mais antigos, inverter o calculo

Popularidade: maximiza ou minimiza popularidade para cada profissao


In [35]:
popularity_by_occupation_df = (
    data.join(user, on="userId", lsuffix="_l", rsuffix="_r")
    .groupby(["occupation", "movieId"]).size()
    .reset_index()
    .rename(columns={0: 'popularity'})
)

In [36]:
popularity_by_occupation_df

,occupation,movieId,popularity
0,administrator,1,37
1,administrator,2,14
2,administrator,3,6
3,administrator,4,20
4,administrator,5,8
...,...,...,...
19285,writer,1608,1
19286,writer,1615,2
19287,writer,1622,1
19288,writer,1646,1


In [37]:
def jaccard(list1, list2):
    union_s = len(set(list1).union(set(list2)))
    inter_s = len(set(list1) & set(list2))

    if (union_s > 0):
        return inter_s/union_s
    else:
        return 0

In [38]:
def gender_sim(df, movie, req):
    """
        Função que calcula a similaridade entre os generos de um filme e um requisito,
        definindo os genêros desejados pelo usuário
    """

    movie_genres = get_genres(df, movie)
    return jaccard(req, movie_genres)

In [39]:
def sim_min_max(df, movie, req, maximize=True):
    """
        Função de similaridade que visa maximizar ou minimizar
        o valor de um requisito definito por req.

        Exemplo: se req='Year' e maximize=True, 
        maximiza diferenca entre o ano de lançamento
        do filme e o valor mínimo, valorizando filmes antigos.

        Se req='year' e maximize=False, maximiza a diferença entre o ano
        de lançamento do filme e o valor máximo. Valorizando assim filmes
        recentes
    """

    target_attributes = df[req]
    max_val = target_attributes.max()
    min_val = target_attributes.min()
    movie_df = df[df["movieId"] == movie]
    if (max_val == min_val or movie_df.empty):
        return 0
    item_attribute = movie_df[req].item()
    if (maximize):
        return (item_attribute - min_val)/(max_val - min_val)
    else:
        return (item_attribute - max_val)/(max_val - min_val)

In [40]:
def sim_popularity_by_occupation(df, movie, req, most_popular=True):
    """
        Função de similaridade que visa maximizar ou minimizar 
        a popularidade de um filme, dado o requisito da ocupação profissional
        de quem assistiu o filme.

        Se most_popular=True, valoriza mais filmes populares para o requisito
        Se most_popular=False, valoriza filmes menos populares para o requisito
    """

    popularity_in_occupation = df[df["occupation"] == req]
    if (len(popularity_in_occupation) == 0):
        return 0

    if (most_popular):
        return sim_min_max(df=popularity_in_occupation, movie=movie, req='popularity', maximize=True)
    else:
        return sim_min_max(df=popularity_in_occupation, movie=movie, req='popularity', maximize=False)
        



In [41]:
sim_popularity_by_occupation(popularity_by_occupation_df, 1, "administrator", most_popular=False)

-0.2653061224489796

In [42]:
gender_sim(df_meta, 1, ["Comedy"])

0.3333333333333333

In [43]:
sim_min_max(item, 1, "year", maximize=False)

-0.039473684210526314

In [98]:
sim_min_max(item, 1, "year", maximize=True)

0.9605263157894737

In [44]:
def knowledge_based_recommendation(movie, most_recent, most_popular, genre, occupation, weights):
    """
        Dadas as restrições most_recent, most_popular, genre e occupation, calcula a similaridade do filme
        movie, ponderando casa similaridade pelo vetor de pesos weights.

        Convenciono que weights é um vetor com 3 pesos entre 0 e 1, onde:

        O 0-esimo peso controla a similaridade por genero de filme
        O 1-esimo peso controla a similaridade por recencia
        O 2-esimo peso controla a similaridade por popularidade em ocupação
    """

    weight_sum = np.sum(weights)

    assert weight_sum == 1

    score = (
            weights[0] * gender_sim(df_meta, movie, genre) +
            # Se most_recent -> minimizamos; Do contrario, maximizamos. Por isso tomamos a negação de most_recent
            weights[1] * sim_min_max(item, movie, "year", maximize = not most_recent) + 
            weights[2] * sim_popularity_by_occupation(popularity_by_occupation_df, movie, occupation, most_popular=most_popular)
    )

    return movie, score/weight_sum

    

    

In [45]:
knowledge_based_recommendation(movie=1, most_recent=True, most_popular=True, genre=['Comedy', 'Animation'], occupation='administrator', weights=[1/3, 1/3, 1/3])

(1, 0.4539622866690536)

In [46]:
def validate_requisites(genres, occupation):
    print("Validando requisitos...")
    try:
        for genre in genres:
            assert genre in genre_list
    except AssertionError:
        print("Um dos generos não é conhecido")
        return False
    try:
        assert occupation in occupation_list
    except AssertionError:
        print("A ocupação profissional informada não é conhecida")
        return False
    print("Requisitos validos!")
    return True

In [47]:
validate_requisites(["Comedy","Açaum"], "administrator")

Validando requisitos...
Um dos generos não é conhecido


False

In [48]:
validate_requisites(["Comedy", "Action"], "administrator")

Validando requisitos...
Requisitos validos!


True

In [49]:
def generate_recommendations(movies_df, most_recent, most_popular, genre, occupation, weights):
    """
        Gera 10 recomendações baseado nos requisitos especificados pelo usuário
    """
    are_valid = validate_requisites(genre, occupation)
    if (not are_valid):
        return
    movies = list(movies_df.movieId.unique())

    recs = []
    for movie in movies:
        movie, score = knowledge_based_recommendation(movie, most_recent, most_popular, genre, occupation, weights)
        recs.append((movie, score))
    sorted_recs = sorted(recs, key=lambda rec: rec[1], reverse=True)

    return pd.DataFrame(sorted_recs[:10], columns=["movieId", "score"]).merge(df_meta[["movieId", "title"]], on="movieId").drop_duplicates()
    
    

In [50]:
generate_recommendations(df_meta, most_recent=True, most_popular=True, genre=['Drama'], occupation='writer', weights=[1/3, 1/3, 1/3]) 

Validando requisitos...
Requisitos validos!


,movieId,score,title
0,9,0.451489,Dead Man Walking (1995)
1,475,0.445774,Trainspotting (1996)
2,15,0.425572,Mr. Holland's Opus (1995)
3,508,0.425572,"People vs. Larry Flynt, The (1996)"
4,58,0.406699,Quiz Show (1994)
5,137,0.405369,Big Night (1996)
6,258,0.404705,Contact (1997)
8,292,0.399654,Rosewood (1997)
9,285,0.395268,Secrets & Lies (1996)
10,86,0.392212,"Remains of the Day, The (1993)"


***Exercício 02:*** Considere um CSP definido como uma tripla (V, D, C) onde:
- V = V_usuario U V_filme : variáveis relacionadas com o usuário e o filme, respectivamente
- C = C_usuario U C_filme U C_ui: restrições do usuário, filme e compatibilidade usuário/filme
- D = conjunto de domínios finitos para as variáveis

Podemos definir as variáveis e restrições como:
- V_usuario : {companhia(sozinho, casal, familia), filme_popular(sim, nao), filme_classico(sim, nao)}
- V_filme : {ano_lancamento(int), generos(list), n_avaliacoes(int)}
- C_usuario : {companhia=casal -> filme_popular=sim ^ companhia=sozinho -> filme_classico=nao}
- C_filme : {lista de itens do catalogo}
- C_ui : {companhia=casal -> generos=[Romance, Drama, Comedy], companhia=familia -> generos=[Children's], filme_classico=sim -> ano_lancamento < 1990, filme_classico=nao -> ano_lancamento > 1993, filme_popular=sim -> n_avaliacoes > 100, companhia=sozinho -> generos=[Horror, War]}

Dada a requisição: REQ={companhia=sozinho, filme_popular=sim}, recomende uma lista de 10 filmes para o usuário.

* De C_usuario: companhia=sozinho -> filme_classico=nao;
* De C_ui: filme_popular=sim -> n_avaliacoes > 100; companhia=sozinho -> generos=[Horror, War]; filme_classico=nao -> ano_lancamento > 1993

Sendo assim, o sistema deve recomendar filmes com mais de 100 avaliacoes, apos 1993 e de Generos Horror e War

In [58]:
# Filmes de genero Horror e War
horror_war_movies = df_meta[df_meta["genre"].isin(["Horror", "War"])]

In [96]:
# Selecionar os filmes com mais de 100 avaliacoes
num_ratings_per_movie = data.groupby("movieId").size().reset_index().rename(columns={0: "frequency"})
df_with_num_ratings = horror_war_movies.merge(num_ratings_per_movie[["movieId", "frequency"]], on="movieId")
popular_movies_by_year = df_with_num_ratings[df_with_num_ratings["frequency"] > 100].merge(item[["movieId", "year"]], on="movieId")

In [90]:
# Filmes lancados apos 1995
possible_recs = popular_movies_by_year[popular_movies_by_year["year"] > 1995]

In [97]:
# Amostra aleatoriamente 10 entradas, já que não definimos uma metodologia de seleção dos 10 melhores candidatos
possible_recs.sample(10)

,movieId,title,genre,frequency,year
21,121,Independence Day (ID4) (1996),War,429,1996
9,288,Scream (1996),Horror,478,1996
11,343,Alien: Resurrection (1997),Horror,124,1997
41,471,Courage Under Fire (1996),War,221,1996
27,181,Return of the Jedi (1983),War,507,1997
0,123,"Frighteners, The (1996)",Horror,115,1996
46,515,"Boot, Das (1981)",War,201,1997
53,879,"Peacemaker, The (1997)",War,136,1997
36,245,"Devil's Own, The (1997)",War,240,1997
15,895,Scream 2 (1997),Horror,106,1997
